### Mortgage Calculations task

This is the first task that is pretty much done - the next few tasks I'll send over whatsapp for you based on what the client sent

1. From the google sheets columns (sent send.xlsx) we need to produce the json output (craft.my.pdf.txt) using Python so the google sheets is not needed
2. For now everything works for the first sheet (assumptions are fixed value with attribute prefixes aspt...)
3. Next task is to incorporate the bank interests rates tab which I'll send over Whatsapp for more task
4. The next task after this is a video I'll send - something about downpayment and no downpayment, then categorising users using it

In [1]:
import pandas as pd
import json
import numpy as np
import numpy_financial as npf
import re
from dataclasses import dataclass

In [2]:
df = pd.read_excel('send.xlsx',sheet_name='For integration V02')

/Users/tengchenghoo/opt/anaconda3/envs/mortgageAutomation/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [3]:
with open('craft.my.pdf.txt','r') as f:
    file = json.load(f)
del file['pdf_output']['pdf_report']

### BOTTOM IS IMPORTANT, DON'T COMMENT OUT

In [4]:
headers = df.iloc[2].values[:341]
headers = ['' if pd.isna(x) else x for x in headers ] 
headers = [re.sub(r'[^\w\s]|(?<=\s)_', '', i) for i in headers if type(i)==str]
match_headers = [i.strip().lower().replace(' ','_') for i in headers if type(i)==str]
match_headers[match_headers.index('supportednot_supported')] = 'supported_not_supported'
match_headers[match_headers.index('additional_upfront')] = 'additional_up_front'

In [ ]:
### checking if you have all the items in the txt file

In [7]:
for k,v in file['input'].items():
    if k not in match_headers:
        print(k)

In [8]:
for k,v in file['flow_output'].items():
    if k not in match_headers:
        print(k)

In [9]:
for k,v in file['pdf_output'].items():
    if k not in match_headers:
        print(k)

In [12]:
### These are the inputs that aren't inside the 'inputs' key in the txt file
required_inputs = ['aff_max_dp_interest','aff_max_dp_pmt_calc', 'aff_max_dp_down_pmt',
                   'available_salary','aff_max_dp_total_paid_amt','debt_years_conf','max_debt',
                  'مبلغ_الدعم', ]

Everything for the Output key values in dict

In [13]:
@dataclass
class OutputFlow:
    aspt_g3 : int = 800000
    aspt_g4 : float = 0.05
    aspt_c2 : int = 70
    aspt_c4 : float = 0.1
    aspt_c6 : int = 30

    def additional_up_front(self,**kwargs):
        if kwargs.get('home_price') > kwargs.get('aff_max_dp_max_mortgage_loan_amt'):
            if kwargs.get('home_price')-kwargs.get('aff_max_dp_max_mortgage_loan_amt') > kwargs.get('debt_qualified'):
                return kwargs.get('home_price')-kwargs.get('aff_max_dp_max_mortgage_loan_amt')
            else:
                return kwargs.get('home_price')-kwargs.get('aff_max_dp_max_mortgage_loan_amt')+kwargs.get('debt_qualified')
        else:
            return 0

    def debt_qualified(self,**kwargs):
        if kwargs.get('home_price') <= self.aspt_g3:
            return kwargs.get('home_price')*self.aspt_g4
        else:
            return kwargs.get('home_price')*self.aspt_c4

    def aff_max_dp_no_of_available_years(self,**kwargs):
        if (self.aspt_c2-kwargs.get('age')) > self.aspt_c6:
            return self.aspt_c6
        else:
            return self.aspt_c2-kwargs.get('age')

    def aff_max_dp_max_mortgage_loan_amt(self,**kwargs):
        """
        Calculates the present value of an investment based on the specified arguments.

        Arguments:
        rate -- interest rate per period
        nper -- total number of payment periods in an investment
        pmt -- payment made each period
        fv (optional) -- future value or cash balance after the last payment is made (default is 0)
        type (optional) -- specifies when payments are due (0 for end of period, 1 for beginning of period; default is 0)
P: Payment made each period
r: Interest rate per period (usually expressed as a percentage, but entered as a decimal value in the formula)
n: Total number of payment periods
        Returns:
        present value of the investment
        """
        rate = kwargs.get('aff_max_dp_interest')/12
        pmt = -kwargs.get('aff_max_dp_pmt_calc')
        nper = kwargs.get('aff_max_dp_no_of_available_years')*12
        return npf.pv(rate, nper, pmt)
    
    def aff_max_dp_max_debt_amt(self,**kwargs):
        return kwargs.get('aff_max_dp_max_mortgage_loan_amt') + kwargs.get('aff_max_dp_down_pmt')
    
    def aff_max_dp_monthly_pmt(self,**kwargs):
        return kwargs.get('available_salary')
                                
    def aff_max_dp_total_interest(self,**kwargs):
        return kwargs.get('aff_max_dp_total_paid_amt') - kwargs.get('aff_max_dp_max_mortgage_loan_amt')
    
    def aff_max_dp_down_pmt(self,**kwargs):
        if (kwargs.get('home_price') <= self.aspt_g3):
            denom = 1-self.aspt_g4
        else:
            denom = 1-self.aspt_c4                                
        return (kwargs.get('aff_max_dp_max_mortgage_loan_amt')/denom)-kwargs.get('aff_max_dp_max_mortgage_loan_amt')


In [15]:
test_output_flow = OutputFlow()

In [16]:
flow_output_dict = {}

In [17]:
for k,_ in file['flow_output'].items():
    method = getattr(test_output_flow,k)
    res = method(**var_dict)
    flow_output_dict[k] = res

O5 = Home_price

BD5 = Debt Qualified

BX5 = Aff_Max_DP Max Mortgage loan AMT

Assumption G3 = 800000

Assumption G4 = 0.05

Assumption C4 = 0.1

Assumption C2 = 70

Assumption C6 = 30

M5 = Age

BU5 = Aff_Max_DP Interest

BT5 = Aff_Max_DP No. of available years

BW5 = Aff_Max_DP PMT Calc

CD5 = Aff_Max_DP Down PMT

BR5 = Available Salary

BV5 = Aff_Max_DP Total Paid AMT

### Flow output
additional_up_front = IF(O5>BX5,IF(O5-BX5>BD5,O5-BX5,O5-BX5+BD5),0)

debt qualified = IF($O5<=Assumption!$G$3,$O5*Assumption!$G$4,$O5*Assumption!$C$4)

Aff_Max_DP No. of available years = IF((Assumption!$C$2-M5)>Assumption!$C$6,Assumption!$C$6,Assumption!$C$2-M5)

aff_max_dp_max_mortgage_loan_amt = PV(BU5/12,BT5*12,-BW5)

Aff_Max_DP Max Debt AMT = BX5+CD5

Aff_Max_DP Monthly PMT = BR5

Aff_Max_DP Total Interest = BV5-BX5

Aff_Max_DP Down PMT = IF($O5<=Assumption!$G$3,(BX5/(1-Assumption!$G$4))-BX5,(BX5/(1-Assumption!$C$4))-BX5)

In [23]:
file['flow_output']

{'additional_up_front': '#N/A',
 'debt_qualified': '250,000',
 'aff_max_dp_no_of_available_years': '20',
 'aff_max_dp_max_mortgage_loan_amt': '#N/A',
 'aff_max_dp_max_debt_amt': '#N/A',
 'aff_max_dp_monthly_pmt': '#N/A',
 'aff_max_dp_total_interest': '#N/A',
 'aff_max_dp_down_pmt': '#N/A'}

Everything in the PDF output key value dict (you cant ignore NDP values for now)

In [31]:
class PdfOutput:
    def __init__(self,years):
        self.years = years
        self.prefix = f'mor_{self.years}_'
        self.aspt_f10 = 'مستحق للدعم'
        self.aspt_j7 = 20
    
    def supporting_merit(self,**kwargs):
        if kwargs.get('supported_not_supported') == self.aspt_f10:
            return kwargs.get('تاكيد_الحسبة')
        else:
            return 0
        
    def aff_max_dp_interest(self,**kwargs):
        return 0.0394
        
    def aff_max_dp_total_paid_amt(self,**kwargs):
        return (kwargs.get('available_salary')*12*kwargs.get('debt_years_conf'))+((kwargs.get('aff_max_dp_no_of_available_years')-kwargs.get('debt_years_conf'))*kwargs.get('salary')*kwargs.get('max_debt')*12)
        
    def _dp_no_of_available_years(self,**kwargs):
        return self.years
    
    def _dp_pv_pmt(self,dp_interest,dp_no_avai_years,dp_pv_debt):
        rate = dp_interest/12
        num_periods = dp_no_avai_years*12
        present_value = dp_pv_debt
        pmt = npf.pmt(rate, num_periods, -present_value)
        return pmt
    
    def _dp_interest(self,**kwargs):
        interest_dict = {
            10 : 0.0394,
            15 : 0.0394,
            20 : 0.0394,
            25 : 0.0394,
            'max' : 0.0394
        }
        
        
        return interest_dict[self.years]
        
    def _dp_principle_amt(self,**kwargs):
        return kwargs.get('home_price')-kwargs.get('additional_up_front')
        
    def _dp_debt_amt(self,**kwargs):
        rate = kwargs.get(self.prefix+'dp_interest')/12
        nper = kwargs.get(self.prefix+'dp_no_of_available_years')*12
        pmt = -kwargs.get(self.prefix+'dp_monthly_pmt')
        return npf.pv(rate, nper, pmt)
        
    def _dp_monthly_pmt(self,**kwargs):
        if self.years == 10:
            pv_debt = kwargs.get(self.prefix+'dp_debt_assumption')
        else:
            pv_debt = kwargs.get(self.prefix+'dp_pv_debt')
        dp_pv_pmt = self._dp_pv_pmt(kwargs.get(self.prefix+'dp_interest'),kwargs.get(self.prefix+'dp_no_of_available_years'),pv_debt)
        return np.min([dp_pv_pmt,kwargs.get('available_salary')])
        
    def _dp_total_paid_amt(self,**kwargs):
        rate = 0
        nper = kwargs.get(self.prefix+'dp_no_of_available_years')*12
        if kwargs.get(self.prefix+'dp_scenario_or_aff') == "Scenario":
            if self.years == 10:
                pv_debt = kwargs.get(self.prefix+'dp_debt_assumption')
            else:
                pv_debt = kwargs.get(self.prefix+'dp_pv_debt')
            dp_pv_pmt = self._dp_pv_pmt(kwargs.get(self.prefix+'dp_interest'),kwargs.get(self.prefix+'dp_no_of_available_years'),pv_debt)
            pmt = -dp_pv_pmt
        else:
            pmt = -kwargs.get(self.prefix+'dp_monthly_pmt')

            
        fv = npf.fv(rate, nper, pmt, 0)
        return fv
    
    def _dp_total_interest(self,**kwargs):
        return kwargs.get(self.prefix+'dp_total_paid_amt')-kwargs.get(self.prefix+'dp_debt_amt')
        
    def _dp_down_pmt(self,**kwargs):
        return kwargs.get(self.prefix+'dp_old_dp')+kwargs.get(self.prefix+'dp_scenario_dp')
        
    def _dp_age_qualification(self,**kwargs):
        if kwargs.get(self.prefix+'dp_no_of_available_years') > kwargs.get('aff_max_dp_no_of_available_years'):
            return "غير مؤهل"            
        return "مؤهل"
                      
    def _dp_pmt_qualification(self,**kwargs):
        if ((kwargs.get(self.prefix+'dp_monthly_pmt')+kwargs.get('other_debt'))/kwargs.get('salary')) <= kwargs.get('max_debt'):
            return "مؤهل"
        return "غير مؤهل"
        
        
    def _dp_pmt(self,**kwargs):
        return (kwargs.get(self.prefix+'dp_monthly_pmt')+kwargs.get('other_debt'))/kwargs.get('salary')
        
    def _dp__of_supported_years(self,**kwargs):
        if kwargs.get(self.prefix+'dp_no_of_available_years') < self.aspt_j7:
            return kwargs.get(self.prefix+'dp_no_of_available_years')
        return self.aspt_j7
        
    def _dp_monthly_support_and_principle(self,**kwargs):
        rate = kwargs.get(self.prefix+'dp_interest')/12
        nper = kwargs.get(self.prefix+'dp_no_of_available_years')*12
        pv = -kwargs.get('مبلغ_الدعم')
        pmt = npf.pmt(rate, nper, pv)
        return pmt
        
    def _dp_support_and_interest(self,**kwargs):
        rate = 0
        nper = kwargs.get(self.prefix+'dp__of_supported_years')*12
        pmt = -kwargs.get(self.prefix+'dp_monthly_support_and_principle')
        pv = 0
        fv = npf.fv(rate, nper, pmt, pv)
        return fv
                      
    def _dp_full_interest_to_cover(self,**kwargs):
        return kwargs.get(self.prefix+'dp_support_and_interest')-kwargs.get('مبلغ_الدعم')
        
    def _dp_covered_interest(self,**kwargs):
        return kwargs.get(self.prefix+'dp_full_interest_to_cover')*kwargs.get('supporting_merit')
        
    def _dp_monthly_support(self,**kwargs):
        return kwargs.get(self.prefix+'dp_covered_interest')/(kwargs.get(self.prefix+'dp__of_supported_years')*12)
        
    def _dp_monthly_pmt_after_support(self,**kwargs):
        return kwargs.get(self.prefix+'dp_monthly_pmt')-kwargs.get(self.prefix+'dp_monthly_support')
        
    def _dp_total_paid_amt_after_support(self,**kwargs):
        return kwargs.get(self.prefix+'dp_total_paid_amt')-kwargs.get(self.prefix+'dp_covered_interest')

mor_10_ndp_no_of_available_years = self.years
mor_10_ndp_interest = 
ARRAY_CONSTRAIN(ARRAYFORMULA(INDEX(Assumption!$C$11:$I$36,MATCH(IM5,Assumption!$B$11:$B$36,0),MATCH($Q5,Assumption!$C$10:$I$10,0))), 1, 1)
mor_10_ndp_principle_amt = 
mor_10_ndp_debt_amt = 
mor_10_ndp_monthly_pmt = 
mor_10_ndp_total_paid_amt = 
mor_10_ndp_total_interest = 
mor_10_ndp_down_pmt = 
mor_10_ndp_personal_loan_interest = 
mor_10_ndp_no_of_years = 
mor_10_ndp_monthly_pl_pmt = 
mor_10_ndp_total_pl_paid_amt = 
mor_10_ndp_total_pl_interest = 
mor_10_ndp_total_monthly_pmt = 
mor_10_ndp_age_qualification = 
mor_10_ndp_pmt_qualification = 
mor_10_ndp_pmt = 
mor_10_ndp__of_supported_years = 
mor_10_ndp_monthly_support_and_principle = 
mor_10_ndp_support_and_interest = 
mor_10_ndp_full_interest_to_cover = 
mor_10_ndp_covered_interest = 
mor_10_ndp_monthly_support = 
mor_10_ndp_monthly_pmt_after_support = 
mor_10_ndp_total_paid_amt_after_support = 

In [34]:
input_dict = {}
for k, _ in file['input'].items():
    input_dict[k] = var_dict[k]

In [35]:
pdf_dict = {}

In [36]:
test_pdf = PdfOutput(10)
for k,_ in file['pdf_output'].items():
    if 'mor_' not in k:
        method = getattr(test_pdf,k)
        res = method(**var_dict)
        pdf_dict[k] = res

In [40]:
def grab_pdf_results(pdf_dict,var_dict,years):
    pdf_obj = PdfOutput(years)
    for k,_ in file['pdf_output'].items():
        if f'mor_{years}' in k and 'ndp' not in k:
            k_suffix = k.split(f'mor_{years}')[1]
            method = getattr(pdf_obj,k_suffix)
            res = method(**var_dict)
            pdf_dict[k] = res
    return pdf_dict

In [41]:
pdf_dict = grab_pdf_results(pdf_dict,var_dict,10)

/Users/tengchenghoo/opt/anaconda3/envs/mortgageAutomation/lib/python3.10/site-packages/numpy_financial/_financial.py:130: RuntimeWarning: invalid value encountered in divide
  (1 + rate*when)*(temp - 1)/rate)


In [42]:
pdf_dict = grab_pdf_results(pdf_dict,var_dict,15)

In [43]:
pdf_dict = grab_pdf_results(pdf_dict,var_dict,20)

In [44]:
pdf_dict = grab_pdf_results(pdf_dict,var_dict,25)

In [45]:
pdf_dict = grab_pdf_results(pdf_dict,var_dict,'max')

In [46]:
pdf_dict

{'supporting_merit': 0.7999999999999999,
 'aff_max_dp_interest': 0.0394,
 'aff_max_dp_total_paid_amt': 4680000.0,
 'mor_10_dp_no_of_available_years': 10,
 'mor_10_dp_interest': 0.0394,
 'mor_10_dp_principle_amt': 1100000,
 'mor_10_dp_debt_amt': 989999.9999999935,
 'mor_10_dp_monthly_pmt': 9995.062514494035,
 'mor_10_dp_total_paid_amt': 1199407.5017392843,
 'mor_10_dp_total_interest': 209407.50173928554,
 'mor_10_dp_down_pmt': 110000.00000000931,
 'mor_10_dp_age_qualification': 'مؤهل',
 'mor_10_dp_pmt_qualification': 'مؤهل',
 'mor_10_dp_pmt': 0.5997531257246984,
 'mor_10_dp__of_supported_years': 10,
 'mor_10_dp_monthly_support_and_principle': 5048.011370956583,
 'mor_10_dp_support_and_interest': 605761.3645147859,
 'mor_10_dp_full_interest_to_cover': 105761.36451478594,
 'mor_10_dp_covered_interest': 84609.09161182874,
 'mor_10_dp_monthly_support': 705.0757634319062,
 'mor_10_dp_monthly_pmt_after_support': 9289.98675106206,
 'mor_10_dp_total_paid_amt_after_support': 1114798.4101274475,


In [48]:
for k,v in var_dict.items():
    if k in var_dict and k in pdf_dict:
        print(k,var_dict[k],pdf_dict[k])
        print('\n')

supporting_merit 0.7999999999999999 0.7999999999999999


aff_max_dp_interest 0.0394 0.0394


aff_max_dp_total_paid_amt 4680000 4680000.0


mor_10_dp_no_of_available_years 10 10


mor_10_dp_interest 0.0394 0.0394


mor_10_dp_principle_amt 1100000 1100000


mor_10_dp_debt_amt 989999.9999999906 989999.9999999935


mor_10_dp_monthly_pmt 9995.062514493968 9995.062514494035


mor_10_dp_total_paid_amt 1199407.501739276 1199407.5017392843


mor_10_dp_total_interest 209407.50173928554 209407.50173928554


mor_10_dp_down_pmt 110000.00000000931 110000.00000000931


mor_10_dp_age_qualification مؤهل مؤهل


mor_10_dp_pmt_qualification مؤهل مؤهل


mor_10_dp_pmt 0.5997531257246984 0.5997531257246984


mor_10_dp__of_supported_years 10 10


mor_10_dp_monthly_support_and_principle 5048.011370956549 5048.011370956583


mor_10_dp_support_and_interest 605761.3645147859 605761.3645147859


mor_10_dp_full_interest_to_cover 105761.36451478594 105761.36451478594


mor_10_dp_covered_interest 84609.09161182874 84

In [49]:
pdf_dict

{'supporting_merit': 0.7999999999999999,
 'aff_max_dp_interest': 0.0394,
 'aff_max_dp_total_paid_amt': 4680000.0,
 'mor_10_dp_no_of_available_years': 10,
 'mor_10_dp_interest': 0.0394,
 'mor_10_dp_principle_amt': 1100000,
 'mor_10_dp_debt_amt': 989999.9999999935,
 'mor_10_dp_monthly_pmt': 9995.062514494035,
 'mor_10_dp_total_paid_amt': 1199407.5017392843,
 'mor_10_dp_total_interest': 209407.50173928554,
 'mor_10_dp_down_pmt': 110000.00000000931,
 'mor_10_dp_age_qualification': 'مؤهل',
 'mor_10_dp_pmt_qualification': 'مؤهل',
 'mor_10_dp_pmt': 0.5997531257246984,
 'mor_10_dp__of_supported_years': 10,
 'mor_10_dp_monthly_support_and_principle': 5048.011370956583,
 'mor_10_dp_support_and_interest': 605761.3645147859,
 'mor_10_dp_full_interest_to_cover': 105761.36451478594,
 'mor_10_dp_covered_interest': 84609.09161182874,
 'mor_10_dp_monthly_support': 705.0757634319062,
 'mor_10_dp_monthly_pmt_after_support': 9289.98675106206,
 'mor_10_dp_total_paid_amt_after_support': 1114798.4101274475,


In [50]:
flow_output_dict

{'additional_up_front': 0,
 'debt_qualified': 110000.0,
 'aff_max_dp_no_of_available_years': 30,
 'aff_max_dp_max_mortgage_loan_amt': 2742831.8360724957,
 'aff_max_dp_max_debt_amt': 3047590.928969434,
 'aff_max_dp_monthly_pmt': 11000,
 'aff_max_dp_total_interest': 1937168.1639275094,
 'aff_max_dp_down_pmt': 304759.09289694345}

In [51]:
input_dict

{'mobile_number': nan,
 'first_name': 'test',
 'last_name': 'test',
 'nationality': 'سعودي',
 'national_id': 9999999991,
 'employer': 'test',
 'desirable_city': 'الرياض',
 'customer_bank_name': 'الاهلي',
 'customer_partner_bank_name': "co-borrower's bank name",
 'salary': 20000,
 'other_debt': 2000,
 'age': 30,
 'years_to_borrow_over': 27,
 'home_price': 1100000,
 'home_searching_status': 'في طور البحث',
 'supported_not_supported': 'مستحق للدعم',
 'working_sector': 'حكومي مدني',
 'is_this_your_first_time_purchasing_home': 'نعم',
 'no_of_people_applying_for_this_mportgage': 'انا فقط',
 'do_you_have_down_payment': 'نعم',
 'interested_area_of_the_city': 'الشمال',
 'family_members': 3,
 'whatsapp_name': nan,
 'advert_filter': nan,
 'action_feedback': nan,
 'action_reasoning': nan,
 'subscription_status': nan,
 'advertiser_name': nan,
 'advertiser_city': nan,
 'advertiser_number': nan,
 'user_consent': nan}

In [52]:
for i in input_dict.keys():
    if pd.isnull(input_dict[i]):
        input_dict[i] = "#N/A"
    if type(i) in [float,int]:
        if i < 1 and i > 0:
            input_dict[i] = str(round(input_dict[i]*100,2))+'%'
    if type(input_dict[i]) in [float,np.float64]:
        input_dict[i] = np.around(input_dict[i],decimals=2)

In [53]:
with open('input.json', 'w', encoding='utf-8') as f:
    json.dump(input_dict, f, ensure_ascii=False)

In [54]:
for i in flow_output_dict.keys():
    if pd.isnull(flow_output_dict[i]):
        flow_output_dict[i] = "#N/A"
    if type(i) in [float,int]:
        if i < 1 and i > 0:
            flow_output_dict[i] = str(round(flow_output_dict[i]*100,2))+'%'
    if type(flow_output_dict[i]) in [float,np.float64]:
        flow_output_dict[i] = np.around(flow_output_dict[i],decimals=2)

In [55]:
for i in pdf_dict.keys():
    if pd.isnull(pdf_dict[i]):
        pdf_dict[i] = "#N/A"
    if type(pdf_dict[i]) in [float,int]:
        if pdf_dict[i] < 1 and pdf_dict[i] > 0:
            pdf_dict[i] = str(round(pdf_dict[i]*100,2))+'%'
    if type(pdf_dict[i]) in [float,np.float64]:
        pdf_dict[i] = np.around(pdf_dict[i],decimals=2)

In [56]:
file_dict = {}
file_dict['input'] = input_dict
file_dict['flow_output'] = flow_output_dict
file_dict['pdf_output'] = pdf_dict

In [57]:
with open('test.json', 'w', encoding='utf-8') as f:
    json.dump(file_dict,f, ensure_ascii=False)

In [58]:
pdf_dict

{'supporting_merit': '80.0%',
 'aff_max_dp_interest': '3.94%',
 'aff_max_dp_total_paid_amt': 4680000.0,
 'mor_10_dp_no_of_available_years': 10,
 'mor_10_dp_interest': '3.94%',
 'mor_10_dp_principle_amt': 1100000,
 'mor_10_dp_debt_amt': 990000.0,
 'mor_10_dp_monthly_pmt': 9995.06,
 'mor_10_dp_total_paid_amt': 1199407.5,
 'mor_10_dp_total_interest': 209407.5,
 'mor_10_dp_down_pmt': 110000.0,
 'mor_10_dp_age_qualification': 'مؤهل',
 'mor_10_dp_pmt_qualification': 'مؤهل',
 'mor_10_dp_pmt': '59.98%',
 'mor_10_dp__of_supported_years': 10,
 'mor_10_dp_monthly_support_and_principle': 5048.01,
 'mor_10_dp_support_and_interest': 605761.36,
 'mor_10_dp_full_interest_to_cover': 105761.36,
 'mor_10_dp_covered_interest': 84609.09,
 'mor_10_dp_monthly_support': 705.08,
 'mor_10_dp_monthly_pmt_after_support': 9289.99,
 'mor_10_dp_total_paid_amt_after_support': 1114798.41,
 'mor_15_dp_no_of_available_years': 15,
 'mor_15_dp_interest': '3.94%',
 'mor_15_dp_principle_amt': 1100000,
 'mor_15_dp_debt_amt':

In [59]:
var_dict

{'mobile_number': nan,
 'first_name': 'test',
 'last_name': 'test',
 'nationality': 'سعودي',
 'national_id': 9999999991,
 'employer': 'test',
 'desirable_city': 'الرياض',
 'customer_bank_name': 'الاهلي',
 'customer_partner_bank_name': "co-borrower's bank name",
 'salary': 20000,
 'other_debt': 2000,
 'debt_years': nan,
 'age': 30,
 'years_to_borrow_over': 27,
 'home_price': 1100000,
 'home_searching_status': 'في طور البحث',
 'supported_not_supported': 'مستحق للدعم',
 'working_sector': 'حكومي مدني',
 'is_this_your_first_time_purchasing_home': 'نعم',
 'no_of_people_applying_for_this_mportgage': 'انا فقط',
 'do_you_have_down_payment': 'نعم',
 'interested_area_of_the_city': 'الشمال',
 'family_members': 3,
 'interested_neighbourhood': nan,
 'asset_type': nan,
 'type_of_mortgage_product': nan,
 'purpose_of_buying_the_asset': nan,
 'user_first_message': nan,
 'whatsapp_name': nan,
 'advert_filter': nan,
 'action_feedback': nan,
 'action_reasoning': nan,
 'subscription_status': nan,
 'advertis

In [60]:
file_dict

{'input': {'mobile_number': '#N/A',
  'first_name': 'test',
  'last_name': 'test',
  'nationality': 'سعودي',
  'national_id': 9999999991,
  'employer': 'test',
  'desirable_city': 'الرياض',
  'customer_bank_name': 'الاهلي',
  'customer_partner_bank_name': "co-borrower's bank name",
  'salary': 20000,
  'other_debt': 2000,
  'age': 30,
  'years_to_borrow_over': 27,
  'home_price': 1100000,
  'home_searching_status': 'في طور البحث',
  'supported_not_supported': 'مستحق للدعم',
  'working_sector': 'حكومي مدني',
  'is_this_your_first_time_purchasing_home': 'نعم',
  'no_of_people_applying_for_this_mportgage': 'انا فقط',
  'do_you_have_down_payment': 'نعم',
  'interested_area_of_the_city': 'الشمال',
  'family_members': 3,
  'whatsapp_name': '#N/A',
  'advert_filter': '#N/A',
  'action_feedback': '#N/A',
  'action_reasoning': '#N/A',
  'subscription_status': '#N/A',
  'advertiser_name': '#N/A',
  'advertiser_city': '#N/A',
  'advertiser_number': '#N/A',
  'user_consent': '#N/A'},
 'flow_output

In [ ]:
file['pdf_output']

In [ ]:
var_dict

Mor_10_DP Monthly PMT-Mor_10_DP Monthly support
Mor_15_DP Monthly PMT-Mor_15_DP Monthly support

### PDF output
Supporting Merit % = IF($Q5=Assumption!$F$10,BN5,0)

Aff_Max_DP No. of available years = ARRAY_CONSTRAIN(ARRAYFORMULA(INDEX(Assumption!$C$11:$I$36,MATCH($BT5,Assumption!$B$11:$B$36,0),MATCH($Q5,Assumption!$C$10:$I$10,0))), 1, 1)

Aff_Max_DP Total Paid AMT = (BR5*12*BS5)+((BT5-BS5)*J5*BQ5*12)

mor_10_dp_no_of_available_years = 10

Mor_10_DP Interest = ARRAY_CONSTRAIN(ARRAYFORMULA(INDEX(Assumption!$C$11:$I$36,MATCH($BT5,Assumption!$B$11:$B$36,0),MATCH($Q5,Assumption!$C$10:$I$10,0))), 1, 1)

Mor_10_DP Principle AMT =$O5-$BC5

Mor_10_DP debt AMT =PV(DX5/12,DW5*12,(-EA5))

Mor_10_DP Monthly PMT = MIN(EB5,$BR5)

Mor_10_DP Total Paid AMT = IF(ED5="Scenario",FV(0,DW5*12,-EB5),FV(0,DW5*12,-EA5))

Mor_10_DP Total Interest = EE5-DZ5

Mor_10_DP Down PMT = EH5+EI5

Mor_10_DP Age Qualification  = IF(DW5>$BT5,"غير مؤهل","مؤهل")

Mor_10_DP PMT Qualification  = IF((EA5+$K5)/$J5<=$BQ5,"مؤهل","غير مؤهل")

Mor_10_DP PMT % = (EA5+$K5)/$J5

Mor_10_DP # of supported years = IF(DW5<Assumption!$J$7,DW5,Assumption!$J$7)

Mor_10_DP Monthly support and Principle = PMT(DX5/12,DW5*12,-$BI5)

Mor_10_DP Support and interest = FV(0,EM5*12,-EN5)

Mor_10_DP full interest to cover = EO5-$BI5

Mor_10_DP Covered interest = EP5*$BO5

Mor_10_DP Monthly support = EQ5/(EM5*12)

Mor_10_DP Monthly PMT after support = EA5-ER5

Mor_10_DP Total Paid AMT after support = EE5-EQ5



mor_15_dp_no_of_available_years = 15

Mor_15_DP Interest = ARRAY_CONSTRAIN(ARRAYFORMULA(INDEX(Assumption!$C$11:$I$36,MATCH($BT5,Assumption!$B$11:$B$36,0),MATCH($Q5,Assumption!$C$10:$I$10,0))), 1, 1)

Mor_15_DP Principle AMT =$O5-$BC5

Mor_15_DP debt AMT =PV(EV5/12,EU5*12,(-EY5))

Mor_15_DP Monthly PMT =MIN(EZ5,$BR5)

Mor_15_DP Total Paid AMT = IF(FB5="Scenario",FV(0,EU5*12,-EZ5),FV(0,EU5*12,-EY5))

Mor_15_DP Total Interest = FC5-EX5

Mor_15_DP Down PMT = FF5+FG5

Mor_15_DP Age Qualification  = IF(EU5>$BT5,"غير مؤهل","مؤهل")

Mor_15_DP PMT Qualification  = IF((EY5+$K5)/$J5<=$BQ5,"مؤهل","غير مؤهل")

Mor_15_DP PMT % = (EY5+$K5)/$J5

Mor_15_DP # of supported years = IF(EU5<Assumption!$J$7,EU5,Assumption!$J$7)

Mor_15_DP Monthly support and Principle = PMT(EV5/12,EU5*12,-$BI5)

Mor_15_DP Support and interest = FV(0,FK5*12,-FL5)

Mor_15_DP full interest to cover = FM5-$BI5

Mor_15_DP Covered interest = FN5*$BO5

Mor_15_DP Monthly support = FO5/(FK5*12)

Mor_15_DP Monthly PMT after support = EY5-FP5 (EA5-ER5)

Mor_15_DP Total Paid AMT after support = FC5-FO5 (EE5-EQ5)

